# 抓取0daytown的百度链接
## 目的
得到大型软件、工具软件教程的百度盘链接，以后方便学习
申请1个月的会员，然后用BeautifulSoup进行抓取，存入xls文件

## 要点
* 分3步
    抓取所有cat号，找出名称catlist
    抓取/cat=xx界面下的所有网页 pagelist
    抓取/p=xx界面下的链接地址 

* 保存为csv格式，
    类型
    文件名
    文件网页
    百度盘地址
    百度盘密码

* 几个方法：
    * xpath找到需要的地址
    * find("a")[href]
    
* 还有一个方法就是直接对http://www.0daytown.com/?p=XXXX 进行解析，关于类别就是抓取这个页面里的
    找到目前最大的http://www.0daytown.com/?p=107109, (191219)，对这个网页进行解析即可

* 要解决抓取过程中没有链接的情况

* 有防爬虫机制，必须更换cookies

* df.append(data)是错误的，而应该进行赋值 df = df.append(data)。

* #将List转为字符串
        baidulinkstr=';'.join(baidulinklist) 
        
* 下载过程中失去链接的处理方法
    from requests.adapters import HTTPAdapter
    s = requests.Session()
    s.mount('http://', HTTPAdapter(max_retries=3))
    r = s.get('http://www.google.com.hk', timeout=5)
    
* glob模块是最简单的模块之一，内容非常少。用它可以查找符合特定规则的文件路径名。跟使用windows下的文件搜索差不多。查找文件只用到三个匹配符：”*”, “?”, “[]”。”*”匹配0个或多个字符；”?”匹配单个字符；”[]”匹配指定范围内的字符，如：[0-9]匹配数字。

* ReadTimeoutError: 出错后，如何重新连接？
    ConnectionError: HTTPConnectionPool(host='www.0daytown.com', port=80): Read timed out.


* '''
测试HTTP time out 怎么办，如何重新链接
ReadTimeoutError: HTTPConnectionPool(host='www.0daytown.com', port=80): Read timed out.

During handling of the above exception, another exception occurred:

ConnectionError                           Traceback (most recent call last)

ConnectionError: HTTPConnectionPool(host='www.0daytown.com', port=80): Read timed out.
'''
'''
解决方法：
1.使用while循环，遇到网络中断修改pagenumber, 。这里用for循环不可以，因为无法修改循环计数
2.用一级except即可，不要用。注意要将所有的except都包括，如果有遗漏将会报错
'''


In [1]:

import requests
import unittest
from bs4 import BeautifulSoup

import urllib3
from selenium import webdriver
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.action_chains import ActionChains

import socket

import pandas as pd

import lxml

import urllib
import random   #随机生成休眠时间
import time

from requests.adapters import HTTPAdapter

import datetime

In [2]:


User_Agent= [
"Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
        "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
        "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
        "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0"
    ]                                #User_Agent的集合


headers1={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:55.0) Gecko/20100101 Firefox/55.0",
         'Cookie': '__cfduid=d71bfdd08a3066b3a91e22850afd06bff1576705865; PHPSESSID=m42hfkvmhr26d2j6rp9aqs81l6; wordpress_test_cookie=WP+Cookie+check; wordpress_logged_in_d1bdccf8f22a41290379013c104952b6=aaronyinyong%7C1577971950%7Co5ISvGpK8eYBZVUk9PuKzXcgasandHEyNu93pybQ9FK%7C457fb1309b2853366c70b4ad57b418e14fd25873dd2fbbb91a3daa45928f38e2'}
# send_headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36",    "Connection": "keep-alive",    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",    "Accept-Language": "zh-CN,zh;q=0.8"    }


timeout1=30
retrynumber=10

In [3]:
def GetArticleList():
    
    urlcat='http://www.0daytown.com/?cat=31&paged=219'
    
    r = requests.get(urlcat,timeout=timeout1,headers=headers1)
    # r.close()
    # r.text
    # print(r.encoding)

    soup=BeautifulSoup(r.text,'lxml')
    # soup
#     print(soup.prettify())

    if "未找到" in soup.find('title'):
        print(soup.find('title'))
    else:
        print('Found')

    ##########找到链接

    tarticle=soup.find(class_='content').find_all('article')#.find("div").find_all('title')
    # print(tarticle)
    titlelist=[]
    for each in tarticle:
    #     print(each)
        print('\n')
    #     each.find(class_="thumbnail").text
        print(each.find('h2').text)
        print(each.find('h2').find('a')['href'])
    #     titlelist.append(each.find(class_="thumbnail").text)
    #     print(titlelist)
    
    print('\n')
    # columnnumbers=len(titlelist)

In [4]:
'''测试上一步函数'''
# GetArticleList()

'测试上一步函数'

In [5]:
'''从每个网页获取百度链接和密码'''

def GetBaiduLinkPwd(url):
    return 1

In [6]:
#根据http://www.0daytown.com/?p=xxxx获取所有清单
#按页面顺序：1~100,101~200,201~300,.....

# totalpagerange=107109 #191220日0daytown网站的最大页面


def GetListbyPageNumbers(eachBatchNumber, PagesPerBatch):
    
    startpagenumber=(eachBatchNumber-1)*PagesPerBatch+1
    endpagenumber=startpagenumber+PagesPerBatch-1
    
    Columns1=['Category','UploadTime','ArticalName','PageLink','BaiduLink','BaiduPwd']
    
    wholepd=pd.DataFrame(columns=Columns1)
    
    print(wholepd)
    baiduinfolist=[]
    
    iTimeSleepCount=0
    
    pagenumber=startpagenumber
    
    while pagenumber<=endpagenumber:
    
    
#     for pagenumber in range(startpagenumber,endpagenumber+1): '''这里用for循环,当网络错误时无法修改循环值,改为while循环'''
    
    
        currentPageNumber=pagenumber
        
        iTimeSleepCount+=1
        
        #每抓100次休息30秒
        if iTimeSleepCount==100:
            time.sleep(30)
            iTimeSleepCount=0
            
        
        #除第一次以后，每抓一次休息2~5秒
        if pagenumber!=startpagenumber:
            time.sleep(0.5)
        
        urlpage='http://www.0daytown.com/?p=' + str(pagenumber)
        print(urlpage)
#         baiduinfolist+=GetBaiduLinkPwd(urlpage)
        #     urlpage='http://www.0daytown.com/?p=5'
#         urlpage=url
    #     print(urlpage)
        try:
            s = requests.Session()
            s.mount('http://', HTTPAdapter(max_retries=retrynumber))
#             r = s.get('http://www.google.com.hk', timeout=5)
#             try:
            r = s.get(urlpage,timeout=timeout1,headers=headers1)
            
#             except requests.exceptions.ConnectTimeout:
#                 print('NETWORK_STATUS = False')
            
#             except requests.exceptions.Timeout:
#                 print('REQUEST_TIMEOUT = TRUE')
            
#             except requests.exceptions.ConnectTimeout as e:
#                 response = e.response 
#                 print(e)
#                 print(response.status_code) # should print a status code, instead response is still None
            
#             r = requests.get(urlpage,timeout=timeout1,headers=headers1)
            

            soup=BeautifulSoup(r.text,'lxml')
        #     print(soup)
    #         print(soup.prettify())
            # tarticle=soup.find('body').find('section').find('div').find('div').find('article') #.find('section')#.find('div')
            # tarticle=soup.find(class_='erphpdown-content-vip')
            # tarticle=tarticle.find(class_='erphpdown-content-vip')

            # tarticle=soup.find(class_='erphpdown-content-vip')

            categorynamelist=soup.find(class_='breadcrumbs').find_all('a')
            categoryname=''
            for each in categorynamelist:
                categoryname+=each.text + "->"
            print(categoryname)

            articlename=soup.find(class_='muted')
            articlename=articlename.text
            print(articlename)

            uploadtime=soup.find('time')
            uploadtime=uploadtime.text.strip()
            uploadtime=uploadtime.split('(')[1]
            uploadtime=uploadtime.split(')')[0]
            print(uploadtime)

            tarticle=soup.find(class_="article-content").find_all('p')

            baidulinklist=[]
            baidupwdlist=[]

            for each in tarticle:
    #             print(each)

#                 try:
                #找出baidupan链接
                #如'<p>Download 百度云<br>
                # 链接: <a href="https://pan.baidu.com/s/1rJq2hyT0z8A3AtWye5SCnQ" rel="nofollow" class="external" target="_blank" data-original-title="" title="">https://pan.baidu.com/s/1rJq2hyT0z8A3AtWye5SCnQ</a> 提取码: ezqd</p>'
#                     if 'pan.baidu.com' in each.find('a').text:
#     #                     print(each.find('a').text)
#                         baidulinklist.append(each.find('a').text)

                ahreflist=each.find_all('a')
                for each_a in ahreflist:
                    if 'pan.baidu.com' in each_a.text:
#                     print(each.find('a').text)
                        baidulinklist.append(each_a.text)

                #找出baidupan密码  如 ' 提取码: ituk</p> 
                if '提取码:' in each.text: 
#                     print(each.text)
                    tempstr1=each.text.split(' 提取码: ')[1]
                    baidupwd=tempstr1.split('</p>')[0]
#                     print(baidupwd)
                    baidupwdlist.append(baidupwd)
                elif  ' 密码: ' in each.text: 
                    tempstr1=each.text.split(' 密码: ')[1]
                    baidupwd=tempstr1.split('</p>')[0]
#                     print(baidupwd)
                    baidupwdlist.append(baidupwd)


#                 except:
    #                 print('no pan.baidu')
                    #空指令
#                     a=1

    #             if '链接:' in each.text: 
    #                 string1=each.text
    #                 stringlist1=string1.split('链接: ')[1]
    #                 if  '提取码:' in  stringlist1:
    #                     stringlist2=stringlist1.split(' 提取码: ')
    #                 elif '密码' in stringlist1:
    #                     stringlist2=stringlist1.split(' 密码: ')

    #                 baidulinklist.append(stringlist2[0])
    #                 baidupwdlist.append(stringlist2[1])

    #             elif '百度云'in each.text:
    #                 print each.text
    #                 stringlist1=each.text.split('链接: ')[1]
    #                 if  '提取码:' in  stringlist1:
    #                     stringlist2=stringlist1.split(' 提取码: ')
    #                 elif '密码' in stringlist1:
    #                     stringlist2=stringlist1.split(' 密码: ')

    #                 baidulinklist.append(each.text)
    #                 baidupwdlist.append(stringlist2[1])


            
#             print()

            #新加一行
    #         insertRow = pd.DataFrame([[categoryname,articlename.text,urlpage,baidulinklist,baidupwdlist]],columns =Columns1)
    #         insertRow = pd.DataFrame([[categoryname,articlename.text,urlpage,baidulinklist,baidupwdlist],columns =Columns1)

            #将List转为字符串
            baidulinkstr=';'.join(baidulinklist)
            baidupwdstr=';'.join(baidupwdlist)

            print(baidulinkstr + '   :    ' + baidupwdstr)
                  
            insertRow = pd.DataFrame([[categoryname,uploadtime,articlename,urlpage,baidulinkstr,baidupwdstr]],columns =Columns1)
    #         print(insertRow)
    #         a=[categoryname,articlename.text,urlpage,baidulinklist,baidupwdlist]
            wholepd=wholepd.append(insertRow,ignore_index=True)
            
            pagenumber+=1
        
        except requests.exceptions.ConnectTimeout:
            print('NETWORK_STATUS = False, please wait 10 seconds')
            time.sleep(10)
            pagenumber=currentPageNumber

        except requests.exceptions.Timeout:
            print('REQUEST_TIMEOUT = TRUE, please wait 10 seconds')
            time.sleep(10)
            pagenumber=currentPageNumber

        except:
            print('#############Unknow error, please wait 10 seconds')
            time.sleep(10)
            pagenumber=currentPageNumber
        
        
        time_stamp = datetime.datetime.now()
        print("当前时间：" + time_stamp.strftime('%Y.%m.%d-%H:%M:%S'))
            
#         print(wholepd)
#         wholepd.append(insertRow,ignore_index=True)
        
#         wholepd['Category']=categoryname
#         wholepd['ArticalName']=articlename.text
#         wholepd['PageLink']=urlpage
#         baiduinfolist=[baidulinklist,baidupwdlist]
#         return baiduinfolist
#     downloadfilepathname='c:/0daydown_'+ str(startpagenumber)+  '_' + str(endpagenumber) + '.csv"
    downloadfilepathname='c:/0daydown_{0}_{1}.csv'.format(startpagenumber,endpagenumber)
   
    wholepd.to_csv(downloadfilepathname,encoding='UTF-8-sig')
    return 1

In [ ]:

'''
批处理X次
Status:当前时间：2019.12.25-20:55:53
'''
startBatchNumber=4
BatchNumbers=2
endBatchNumber=startBatchNumber+BatchNumbers

'''每次处理100个网页，保存为一个文件'''
PagesPerBatch=10000

'''按顺序：1~100,101~200,201~300,.....'''
for eachBatchNumber in range(startBatchNumber,endBatchNumber):
    GetListbyPageNumbers(eachBatchNumber, PagesPerBatch)
    
    #每批次休息半分钟
    time.sleep(30)

print('done')

Empty DataFrame
Columns: [Category, UploadTime, ArticalName, PageLink, BaiduLink, BaiduPwd]
Index: []
http://www.0daytown.com/?p=30001
0daytown->Software->Windows->
RS Data Recovery Software 03.2015 Multilingual
2015-12-27
http://pan.baidu.com/s/1ntFWETF   :    
当前时间：2019.12.26-16:42:42
http://www.0daytown.com/?p=30002
0daytown->Software->Windows->系统程序->
Wise Care 365 Pro 3.58 Build 318 Multilingual + Portable 系统优化软件
2015-12-27
http://pan.baidu.com/s/1o6zF886   :    
当前时间：2019.12.26-16:42:45
http://www.0daytown.com/?p=30003
0daytown->Software->Windows->
DAYU Disk Master Professional 2.8
2015-12-27
http://pan.baidu.com/s/1kTCCRXH   :    
当前时间：2019.12.26-16:42:47
http://www.0daytown.com/?p=30004
0daytown->Games->
Life Is Strange Episode 2-CODEX
2015-12-27
http://pan.baidu.com/share/link?shareid=914459559&uk=621312725   :    
当前时间：2019.12.26-16:42:50
http://www.0daytown.com/?p=30005
0daytown->Games->
Survivor Squad v1.16-ALI213
2015-12-27
http://pan.baidu.com/share/link?shareid=3795369996

http://www.0daytown.com/?p=30045
0daytown->教程->
Guitar Play-Along: Volume 15 – Black Sabbath
2015-12-28
http://pan.baidu.com/s/1gdu4nnl   :    4vzh
当前时间：2019.12.26-16:44:30
http://www.0daytown.com/?p=30046
0daytown->素材->音色/Timbre->
Industrykits.com VIP Members Only Exclusive PresetBank V3 KONTAKT
2015-12-28
http://pan.baidu.com/s/1eQzVTYi   :    1h8p
当前时间：2019.12.26-16:44:32
http://www.0daytown.com/?p=30047
0daytown->素材->音色/Timbre->
Industrykits.com VIP Members Only Exclusive PresetBank V4 KONTAKT
2015-12-28
http://pan.baidu.com/s/1hqgUZwo   :    6ftt
当前时间：2019.12.26-16:44:34
http://www.0daytown.com/?p=30048
0daytown->素材->音色/Timbre->
Audiobro – LA Scoring Strings 2.0 16bit Version
2015-12-28
http://pan.baidu.com/s/1dDxWdUD   :    f2dh
当前时间：2019.12.26-16:44:40
http://www.0daytown.com/?p=30049
#############Unknow error, please wait 10 seconds
当前时间：2019.12.26-16:45:22
http://www.0daytown.com/?p=30049
0daytown->教程->
Arts Magic – Play Buddy Guy With Max Milligan
2015-12-28
http://pan.baidu.

http://www.0daytown.com/?p=30090
0daytown->教程->其他教程->
Treehouse – PHP Testing
2015-12-29
http://pan.baidu.com/s/1pJFmBWZ   :    4xur
当前时间：2019.12.26-16:48:26
http://www.0daytown.com/?p=30091
0daytown->教程->网络技术->
CBT Nuggets – Juniper Networks Certified Associate Junos JN0-102 (2014) [repost]
2015-12-29
http://pan.baidu.com/s/1kT452vl   :    7x7s
当前时间：2019.12.26-16:48:28
http://www.0daytown.com/?p=30092
0daytown->教程->
Lynda – WordPress and Genesis DIY: Weddings and Special Events
2015-12-29
http://pan.baidu.com/s/1bn3zQov   :    
当前时间：2019.12.26-16:48:31
http://www.0daytown.com/?p=30093
0daytown->教程->
Lynda – Excel 2011 for the Mac: Managing and Analyzing Data
2015-12-29
http://pan.baidu.com/s/16jKbO   :    ssp6
当前时间：2019.12.26-16:48:33
http://www.0daytown.com/?p=30094
0daytown->教程->其他教程->
Lynda – WordPress DIY: Weddings and Special Events
2015-12-29
http://pan.baidu.com/s/1mgssi1a   :    23y6
当前时间：2019.12.26-16:48:35
http://www.0daytown.com/?p=30095
0daytown->教程->其他教程->
Scott Wilson – 

http://www.0daytown.com/?p=30137
0daytown->教程->吉他技术->
Guitar Play-Along: Volume 23 – Blues Classics
2015-12-30
http://pan.baidu.com/s/1i35p7gx   :    x9bk
当前时间：2019.12.26-16:52:06
http://www.0daytown.com/?p=30138
0daytown->教程->吉他技术->
Guitar Play-Along: Volume 25 – Hard Rock
2015-12-30
http://pan.baidu.com/s/1gdrg0qJ   :    c2pa
当前时间：2019.12.26-16:52:09
http://www.0daytown.com/?p=30139
0daytown->教程->
Guitar Play-Along: Volume 26 – 70s Classics
2015-12-30
http://pan.baidu.com/s/1qWKAsaw   :    tvzu
当前时间：2019.12.26-16:52:12
http://www.0daytown.com/?p=30140
0daytown->教程->其他教程->
Color Theory I: Complementary Colors by Bill Perkins
2015-12-30
http://pan.baidu.com/s/1jGFwTQQ   :    36y5
当前时间：2019.12.26-16:52:15
http://www.0daytown.com/?p=30141
0daytown->教程->
Become An Importer: Source Products & Sell For Huge Profits!
2015-12-30
http://pan.baidu.com/s/1Gtihw   :    spew
当前时间：2019.12.26-16:52:18
http://www.0daytown.com/?p=30142
0daytown->Software->Windows->
WebCruiser Web Vulnerability Scanner

http://www.0daytown.com/?p=30182
0daytown->Software->CAX/EDA->
Autodesk AutoCAD Civil 3D 2016 Win64
2015-12-30
http://pan.baidu.com/s/1dDrk68x   :    rxw2
当前时间：2019.12.26-16:55:51
http://www.0daytown.com/?p=30183
0daytown->教程->
Overcome Social Phobia Using NLP Techniques
2015-12-30
http://pan.baidu.com/s/1jGoY3wM   :    3638
当前时间：2019.12.26-16:55:55
http://www.0daytown.com/?p=30184
0daytown->Ebooks->
Lifestyle For Men – March 2015-P2P
2015-12-31
http://pan.baidu.com/s/1kTuweVl   :    
当前时间：2019.12.26-16:55:57
http://www.0daytown.com/?p=30185
0daytown->Ebooks->
3D Artist – Issue 79, 2015-P2P
2015-12-31
http://pan.baidu.com/s/1kTJyEu7   :    
当前时间：2019.12.26-16:55:59
http://www.0daytown.com/?p=30186
0daytown->Ebooks->
GamesTM – Issue 159, 2015-P2P
2015-12-31
http://pan.baidu.com/s/1dDF7hE5   :    
当前时间：2019.12.26-16:56:01
http://www.0daytown.com/?p=30187
0daytown->Ebooks->
Amateur Photographer – 4 April 2015-P2P
2015-12-31
http://pan.baidu.com/s/1pJ9Nb8R   :    
当前时间：2019.12.26-16:56:03


...

0daytown->教程->其他教程->
Udemy – Laravel Homestead on Windows 10
2016-05-15
http://pan.baidu.com/s/1bPnAI   :    9y8p
当前时间：2019.12.27-01:33:57
http://www.0daytown.com/?p=37401
0daytown->素材->音色/Timbre->
Push Button Bang Rise 2 Beyond Noise FX WAV
2016-05-15
http://pan.baidu.com/s/1nt9YGQ1   :    6k3a
当前时间：2019.12.27-01:33:59
http://www.0daytown.com/?p=37402
0daytown->素材->音色/Timbre->
Freaky Loops Be A Pro EDM Producer MULTiFORMAT
2016-05-15
http://pan.baidu.com/s/1gdi0Coz   :    gyta
当前时间：2019.12.27-01:34:02
http://www.0daytown.com/?p=37403
0daytown->素材->音色/Timbre->
Audio Imperia Photosynthesis Series Vol.1 Sphere KONTAKT
2016-05-15
http://pan.baidu.com/s/1hqHA4YC   :    k95k
当前时间：2019.12.27-01:34:05
http://www.0daytown.com/?p=37404
0daytown->Software->MacOSX->
Klanghelm MJUC variable-tube compressor v1.0.2 [WiN/OSX]
2016-05-15
http://pan.baidu.com/s/1sjr9EdZ   :    
当前时间：2019.12.27-01:34:08
http://www.0daytown.com/?p=37405
0daytown->Software->Windows->
O&O Defrag Professional / Server / Wor

http://www.0daytown.com/?p=37448
0daytown->素材->音色/Timbre->
Future Music – Progressive Rock WAV
2016-05-16
http://pan.baidu.com/s/1qWrBHEk   :    sieb
当前时间：2019.12.27-01:36:31
http://www.0daytown.com/?p=37449
0daytown->素材->音色/Timbre->
Digital Juice Sound FX Library II D01-12 DVDR-CFE
2016-05-16
http://pan.baidu.com/s/1i38Lc81   :    megn
当前时间：2019.12.27-01:36:33
http://www.0daytown.com/?p=37450
0daytown->素材->音色/Timbre->
WA Production What About Ultimate EDM Collection Vol.4 WAV MiDi Sylenth1 SPiRE
2016-05-16
http://pan.baidu.com/s/1c04l18s   :    8thj
当前时间：2019.12.27-01:36:35
http://www.0daytown.com/?p=37451
0daytown->素材->音色/Timbre->
Best Service Hardstyle Samples Vol. 2 ACID WAV REFILL
2016-05-16
http://pan.baidu.com/s/1mgszQZI   :    bxzw
当前时间：2019.12.27-01:36:37
http://www.0daytown.com/?p=37452
0daytown->素材->音色/Timbre->
Big Fish Audio Mahadhi African Rhythms ACID WAV REX AIFF
2016-05-16
http://pan.baidu.com/s/1pJiQt3X   :    6a2k
当前时间：2019.12.27-01:36:39
http://www.0daytown.com/?p=37

http://www.0daytown.com/?p=37494
0daytown->Software->Windows->
Cubify Sculpt 2015
2016-05-17
http://pan.baidu.com/s/1pJiQxcv   :    
当前时间：2019.12.27-01:38:49
http://www.0daytown.com/?p=37495
0daytown->教程->网络技术->
Asp.Net MVC Web App On 3 Tier Architecture
2016-05-17
http://pan.baidu.com/s/1jGjaCuE   :    93pq
当前时间：2019.12.27-01:38:51
http://www.0daytown.com/?p=37496
0daytown->教程->摄影技术->
Lynda – Shooting a Photo Essay: Coney Island
2016-05-17
http://pan.baidu.com/s/1ntsxX9Z   :    vxwm
当前时间：2019.12.27-01:38:53
http://www.0daytown.com/?p=37497
0daytown->教程->其他教程->
Lynda – Bridge CC (2015) Essential Training
2016-05-17
http://pan.baidu.com/s/1c04R5lu   :    
当前时间：2019.12.27-01:38:56
http://www.0daytown.com/?p=37498
0daytown->教程->SolidWorks教程->
Lynda – Modeling a Cabinet with SOLIDWORKS
2016-05-17
   :    
当前时间：2019.12.27-01:38:58
http://www.0daytown.com/?p=37499
0daytown->教程->
Lynda – Up and Running with Audacity (updated Aug 14, 2015)
2016-05-17
http://pan.baidu.com/s/1dD8Thzr   :    ckdq

http://www.0daytown.com/?p=37541
0daytown->教程->
祖絲《何芷晴》[192K/MP3][CT/YF][40M]
2016-05-18
   :    
当前时间：2019.12.27-01:40:53
http://www.0daytown.com/?p=37542
0daytown->教程->
李克勤《我克勤》[320K/MP3][CT/YF][70M]
2016-05-18
   :    
当前时间：2019.12.27-01:40:55
http://www.0daytown.com/?p=37543
0daytown->教程->
群星 -《将世界的心拥在一起》[APE+CUE]
2016-05-18
   :    
当前时间：2019.12.27-01:40:58
http://www.0daytown.com/?p=37544
0daytown->教程->
群星 -《唱戏·八〇年最佳电影电视主题曲》[APE+CUE]
2016-05-18
   :    
当前时间：2019.12.27-01:41:00
http://www.0daytown.com/?p=37545
0daytown->教程->
群星 -《摘下面具与我共舞》[APE+CUE]
2016-05-18
   :    
当前时间：2019.12.27-01:41:03
http://www.0daytown.com/?p=37546
0daytown->教程->
群星 -《1992抒情金曲精选·最爱的歌》[APE+CUE]
2016-05-18
   :    
当前时间：2019.12.27-01:41:06
http://www.0daytown.com/?p=37547
0daytown->Games->
The Sims 4 Spa Day Addon-RELOADED
2016-05-18
http://pan.baidu.com/s/1dD2NDTn   :    76nm
当前时间：2019.12.27-01:41:08
http://www.0daytown.com/?p=37548
0daytown->Software->MacOSX->
Terraria v1.3.0.8 MacOSX-P2P
2016-05-18
htt

0daytown->Software->Windows->
VideoDetach Pro 1.2.7
2016-05-19
http://pan.baidu.com/s/1o63Rpyy   :    
当前时间：2019.12.27-01:43:55
http://www.0daytown.com/?p=37591
0daytown->Software->CAX/EDA->
ANSYS 16.0-16.1 ACT Extentions (build 08.2015)
2016-05-19
http://pan.baidu.com/s/1hqrSkKG   :    takn
当前时间：2019.12.27-01:43:58
http://www.0daytown.com/?p=37592
0daytown->Software->Windows->
MedCalc 15.8
2016-05-19
http://pan.baidu.com/s/1mgvhOgS   :    
当前时间：2019.12.27-01:43:59
http://www.0daytown.com/?p=37593
0daytown->教程->其他教程->
How To Use Photoshop For Beginners The Free and Easy Way
2016-05-19
http://pan.baidu.com/s/1ntkW3WX   :    2sw8
当前时间：2019.12.27-01:44:02


In [ ]:
# Columns1=['Category','ArticalName','PageLink','BaiduLink','BaiduPwd']

# # data={'Category':[1],'ArticalName':[2],'PageLink':[3],'BaiduLink':[4],'BaiduPwd':[5]} 
# wholepd=pd.DataFrame(columns=Columns1)
# # wholepd=  
# print(wholepd)
# print("\n")

# insertRow = pd.DataFrame([[categoryname,articlename.text,urlpage,baidulinklist,baidupwdlist]],columns =Columns1)
# print(insertRow)
# print("\n")
# # a=['categoryname','articlename.text','urlpage','baidulinklist','baidupwdlist']
# # a=[1,2,3,4,5]
# data1={'Category':[3],'ArticalName':[5],'PageLink':[7],'BaiduLink':[8],'BaiduPwd':[9]}

# wholepd=wholepd.append(insertRow,ignore_index=True)
# # wholepd.append([['categoryname','articlename.text','urlpage','baidulinklist','baidupwdlist']],ignore_index=True)

# print(wholepd)

In [18]:
'''
将分散的csv文件合并
https://blog.csdn.net/u012735708/article/details/83989842
'''

import glob
import pandas as pd

# csvPath='C:/Users/Administrator/Documents/MyMobileBooks_800_Reading/0DayFileList/test/'
csvPath='C:/LocalTempFiles/'

Columns1=['Category','UploadTime','ArticalName','PageLink','BaiduLink','BaiduPwd']
header1=['number','Category','UploadTime','ArticalName','PageLink','BaiduLink','BaiduPwd']
outputfile=csvPath + '0daytown_1_1000.csv' 

def CombineCSV():
    
#     for inputfile in os.walk(HtmlSavePath):
    csv_list=glob.glob(csvPath + '*.csv')
                       
    print(u'共发现%s个CSV文件'% len(csv_list))
    
    print(u'正在处理............')
    
    for inputfile in  glob.glob(csvPath + '/*.csv'):
        
#         csvfilenames += [csvPath + name for name in files if name.endswith(".csv")]
                                
        f=open(inputfile,encoding='utf-8')        
        data=pd.read_csv(f)        
        data.to_csv(outputfile,mode='a',index=True,header=None,encoding='utf-8-sig')   
        f.close()
#         data.close()
        
    print('完成合并') 

def CleanDuplicates(file):    
    df = pd.read_csv(file,header=0)   
    datalist = df.drop_duplicates()    
    datalist.to_csv(file)    
    print('完成去重') 
    
if __name__ == '__main__':    
    CombineCSV()    
#     CleanDuplicates(outputfile)

共发现63个CSV文件
正在处理............
完成合并
